# Manage prompts programmatically

## Push a prompt

To create a new prompt or update an existing prompt, you can use the push prompt method.


In [3]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate

In [4]:
client = Client()

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
url = client.push_prompt("joke-generator", object=prompt)
# url is a link to the prompt in the UI
print(url)

https://smith.langchain.com/prompts/joke-generator/b9244726?organizationId=4791d9fe-98f1-47bb-b116-297cd74a3dc0


Alternatively, you can use LangChain

In [5]:
from langchain import hub as prompts

In [7]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
url = prompts.push("joke-generator-2", prompt)
# url is a link to the prompt in the UI
print(url)

https://smith.langchain.com/prompts/joke-generator-2/b9244726?organizationId=4791d9fe-98f1-47bb-b116-297cd74a3dc0


You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [8]:
from langchain_openai import ChatOpenAI

In [9]:
client = Client()
model = ChatOpenAI(model="gpt-4o-mini")

In [10]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

client.push_prompt("joke-generator-with-model", object=chain)

'https://smith.langchain.com/prompts/joke-generator-with-model/0c1faa33?organizationId=4791d9fe-98f1-47bb-b116-297cd74a3dc0'

## Pull a prompt

To pull a prompt, you can use the pull prompt method, which returns a the prompt as a langchain PromptTemplate.

To pull a private prompt you do not need to specify the owner handle (though you can, if you have one set).

To pull a public prompt from the LangChain Hub, you need to specify the handle of the prompt's author.

In [11]:
prompt = client.pull_prompt("joke-generator")
model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model
chain.invoke({"topic": "cats"})

AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'stop', 'logprobs': None}, id='run-611791d6-dc2b-402f-ba20-644df778112c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Or with LangChain

In [12]:
prompt = prompts.pull("joke-generator")
model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model
chain.invoke({"topic": "cats"})

AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-98ef5997-a302-4a73-95d6-312c2a96a10d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Similar to pushing a prompt, you can also pull a prompt as a RunnableSequence of a prompt and a model. Just specify include_model when pulling the prompt. If the stored prompt includes a model, it will be returned as a RunnableSequence. Make sure you have the proper environment variables set for the model you are using.

In [13]:
chain = client.pull_prompt("joke-generator-with-model", include_model=True)
chain.invoke({"topic": "cats"})

AIMessage(content='Why did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac1e992f-8adf-4bc5-9df0-165d5fde6361-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

When pulling a prompt, you can also specify a specific commit hash or prompt tag to pull a specific version of the prompt.

In [15]:
prompt = client.pull_prompt("joke-generator:b9244726")

To pull a public prompt from the LangChain Hub, you need to specify the handle of the prompt's author.

In [16]:
prompt = client.pull_prompt("efriis/my-first-prompt")

In [22]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['profession'], input_types={}, partial_variables={}, template='You are an expert {profession} who loves answering questions cheerfully.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question} '), additional_kwargs={})]

## Use a prompt without LangChain

If you want to store your prompts in LangSmith but use them directly with a model provider's API, you can use our conversion methods. These convert your prompt into the payload required for the OpenAI or Anthropic API.

These conversion methods rely on logic from within LangChain integration packages, and you will need to install the appropriate package as a dependency in addition to your official SDK of choice. Here are some examples:

### OpenAI

In [23]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

In [24]:
# openai client
oai_client = OpenAI()

# pull prompt and invoke to populate the variables
prompt = client.pull_prompt("joke-generator")
prompt_value = prompt.invoke({"topic": "cats"})

openai_payload = convert_prompt_to_openai_format(prompt_value)
openai_response = oai_client.chat.completions.create(**openai_payload)

In [25]:
openai_response

ChatCompletion(id='chatcmpl-Ab60TUYMQic8YbdJi5MnmQgVu7mel', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733404589, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=13, total_tokens=33, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Anthropic

In [26]:
from anthropic import Anthropic
from langsmith.client import convert_prompt_to_anthropic_format

In [27]:
# anthropic client
anthropic_client = Anthropic()
# pull prompt and invoke to populate the variables
prompt = client.pull_prompt("joke-generator")
prompt_value = prompt.invoke({"topic": "cats"})
anthropic_payload = convert_prompt_to_anthropic_format(prompt_value)
anthropic_response = anthropic_client.messages.create(**anthropic_payload)

In [28]:
anthropic_response

Message(id='msg_01UR3mgaqvrVWF4x29dcZUBR', content=[TextBlock(text='Here\'s a cat joke for you:\n\nWhy don\'t cats play poker in the jungle? \nToo many cheetahs!\n\nHow\'s that? Hopefully you got a little chuckle out of the pun on "cheetahs" sounding like "cheaters." Cats and their love of puns - or at least our human attempts at cat-themed puns. Let me know if you\'d like to hear another cat joke.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=13, output_tokens=99))

## List, delete, and like prompts

You can also list, delete, and like/unlike prompts using the list prompts, delete prompt, like prompt and unlike prompt methods. See the LangSmith SDK client for extensive documentation on these methods.

In [29]:
# List all prompts in my workspace
prompts = client.list_prompts()

In [33]:
# List my private prompts that include "joke"
prompts = client.list_prompts(query="joke", is_public=False)

In [34]:
prompts

ListPromptsResponse(repos=[Prompt(repo_handle='joke-generator-with-model', description='', readme='', id='845f5157-b351-46ca-a8d5-4f890a617b46', tenant_id='4791d9fe-98f1-47bb-b116-297cd74a3dc0', created_at=datetime.datetime(2024, 12, 5, 13, 10, 7, 53907), updated_at=datetime.datetime(2024, 12, 5, 13, 10, 8, 186709), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner=None, full_name='joke-generator-with-model', num_likes=0, num_downloads=1, num_views=0, liked_by_auth_user=False, last_commit_hash='0c1faa33da1f14c5669245781eaca359d34a515c6054b9f82252719e87ea74b5', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None), Prompt(repo_handle='joke-generator-2', description='', readme='', id='557f075e-aef5-4b14-be13-d7b45830f7c8', tenant_id='4791d9fe-98f1-47bb-b116-297cd74a3dc0', created_at=datetime.datetime(2024, 12, 5, 13, 9, 9, 144338), updated_at=datetime.datetime(2024, 12, 5, 13, 9, 11, 269719), is_publi

In [35]:
# Delete a prompt
client.delete_prompt("joke-generator-2")

In [36]:
# Like a prompt
client.like_prompt("efriis/my-first-prompt")
# Unlike a prompt
client.unlike_prompt("efriis/my-first-prompt")

{'likes': 1}